# Introduction to Python GPU Programming with Numba

In [1]:
# Misc. import
from __future__ import print_function
from IPython.display import Image 
%matplotlib inline

# Numba
* Opensource BSD license
* Basic CUDA GPU JIT compilation
* OpenCL support coming

In [2]:
import numba
print("numba", numba.__version__)

numba 0.33.0


# The CUDA GPU

- A massively parallel processor (many cores)
    - 100 threads, 1,000 threads, and more
- optimized for data throughput
    - Simple (shallow) cache hierarchy
    - Best with manual caching!
    - Cache memory is called shared memory and it is addressable
- CPU is latency optimized
    - Deep cache hierarchy
    - L1, L2, L3 caches
- GPU execution model is different
- GPU forces you to think and program *in parallel*

In [3]:
# Get all the imports we need
import numba.cuda
import numpy as np
import math

my_gpu = numba.cuda.get_current_device()
print("Running on GPU:", my_gpu.name)
cores_per_capability = {
    1: 8,
    2: 32,
    3: 192,
    5: 128
}
cc = my_gpu.compute_capability
print("Compute capability: ", "%d.%d" % cc, "(Numba requires >= 2.0)")
majorcc = cc[0]
print("Number of streaming multiprocessor:", my_gpu.MULTIPROCESSOR_COUNT)
cores_per_multiprocessor = cores_per_capability[majorcc]
print("Number of cores per mutliprocessor:", cores_per_multiprocessor)
total_cores = cores_per_multiprocessor * my_gpu.MULTIPROCESSOR_COUNT
print("Number of cores on GPU:", total_cores)

Running on GPU: Tesla K40m
Compute capability:  3.5 (Numba requires >= 2.0)
Number of streaming multiprocessor: 15
Number of cores per mutliprocessor: 192
Number of cores on GPU: 2880


# High-level Array-Oriented Style

- Use NumPy array as a unit of computation
- Use NumPy universal function (ufunc) as an abstraction of computation and scheduling
- ufuncs are elementwise functions
- If you use NumPy, you are using ufuncs

In [4]:
print(np.sin, "is of type", type(np.sin))
print(np.add, "is of type", type(np.add))

<ufunc 'sin'> is of type <type 'numpy.ufunc'>
<ufunc 'add'> is of type <type 'numpy.ufunc'>


# Vectorize

- generate a ufunc from a python function
- converts scalar function to elementwise array function
- Numba provides CPU and GPU support

In [5]:
# CPU version
@numba.vectorize(['float32(float32, float32)',
                  'float64(float64, float64)'], target='cpu')
def cpu_sincos(x, y):
    return math.sin(x) * math.cos(y)

# CUDA version
@numba.vectorize(['float32(float32, float32)',
                     'float64(float64, float64)'], target='cuda')
def gpu_sincos(x, y):
    return math.sin(x) * math.cos(y)

```
@numba.vectorize(<list of signatures>, target=<'cpu', 'gpu'>)
```

- A ufunc can be overloaded to work on multiple type signatures


### Test it out

- 2 input arrays
- 1 output array
- 1 million doubles (8 MB) per array
- Total 24 MB of data

In [6]:
# Generate data
n = 1000000
x = np.linspace(0, np.pi, n)
y = np.linspace(0, np.pi, n)

# Check result
np_ans = np.sin(x) * np.cos(y)
nb_cpu_ans = cpu_sincos(x, y)
nb_gpu_ans = gpu_sincos(x, y)

print("CPU vectorize correct: ", np.allclose(nb_cpu_ans, np_ans))
print("GPU vectorize correct: ", np.allclose(nb_gpu_ans, np_ans))

CPU vectorize correct:  True
GPU vectorize correct:  True


## Benchmark

In [7]:
print("NumPy")
%timeit np.sin(x) * np.cos(y)

print("CPU vectorize")
%timeit cpu_sincos(x, y)

print("GPU vectorize")
%timeit gpu_sincos(x, y)

# Optional cleanup 
del x, y

NumPy
10 loops, best of 3: 72.3 ms per loop
CPU vectorize
10 loops, best of 3: 63.7 ms per loop
GPU vectorize
100 loops, best of 3: 17.1 ms per loop


- CPU vectorize time is similar to pure NumPy time because ``sin()`` and ``cos()`` calls dominate the time.
- GPU vectorize is a lot faster



## Behind the scence




### Automatic memory transfer

- NumPy arrays are automatically transferred
    - CPU -> GPU
    - GPU -> CPU


### Automatic work scheduling

- The work is distributed the across all threads on the GPU
- The GPU hardware handles the scheduling



### Automatic GPU memory management

- GPU memory is tied to object lifetime
- freed automatically

# Generalized Universal Function (guvectorize)

- Vectorize is limited to scalar arguments in the core function
- GUVectorize accepts array arguments

In [28]:
@numba.guvectorize(['void(float32[:,:], float32[:,:], float32[:,:])'],
                      '(m, n),(n, p)->(m, p)', target='cuda')
def batch_matrix_mult(a, b, c):
    for i in range(c.shape[0]):
        for j in range(c.shape[1]):
            tmp = 0
            for n in range(a.shape[1]):
                 tmp += a[i, n] * b[n, j]
            c[i, j] = tmp

```
@numba.guvectorize(<list of function signatures>, <a string to desc the shape signature>, target=<'cpu', 'cuda'>)
```

In [29]:
a = np.arange(1.0, 10.0, dtype=np.float32).reshape(3,3)
b = np.arange(1.0, 10.0, dtype=np.float32).reshape(3,3)

# Use the builtin matrix_multiply in NumPy for CPU test
import numpy.core.umath_tests as ut

# Check result
print('NumPy result')
np_ans = ut.matrix_multiply(a, b)
print(np_ans)

print('Numba GPU result')
gpu_ans = batch_matrix_mult(a, b)
print(gpu_ans)

assert np.allclose(np_ans, gpu_ans)


NumPy result
[[  30.   36.   42.]
 [  66.   81.   96.]
 [ 102.  126.  150.]]
Numba GPU result
[[  30.   36.   42.]
 [  66.   81.   96.]
 [ 102.  126.  150.]]


### Test it out

- batch multiply two 4 million 2x2 matrices 

In [30]:
n = 4000000
dim = 2
a = np.random.random(n * dim * dim).astype(np.float32).reshape(n, dim, dim)
b = np.random.random(n * dim * dim).astype(np.float32).reshape(n, dim, dim)

print('NumPy time')
%timeit ut.matrix_multiply(a, b)

print('Numba GPU time')
%timeit batch_matrix_mult(a, b)

NumPy time
10 loops, best of 3: 139 ms per loop
Numba GPU time
1 loop, best of 3: 220 ms per loop


- GPU time seems to be similar to CPU time



## Manually Transfer the data to the GPU

- This will let us see the actual compute time without the CPU<->GPU transfer

In [31]:
dc = numba.cuda.device_array_like(a)
da = numba.cuda.to_device(a)
db = numba.cuda.to_device(b)

* ```numba.cuda.device_array_like``` allocate without initialization with the type and shape of another array.
    * similar to ```numpy.empty_like(a)```
* ```numba.cuda.to_device``` create a GPU copy of the CPU array



## Pure compute time on the GPU

In [32]:
def check_pure_compute_time(da, db, dc):
    batch_matrix_mult(da, db, out=dc)
    numba.cuda.synchronize()   # ensure the call has completed
    
%timeit check_pure_compute_time(da, db, dc)
del da, db, dc

100 loops, best of 3: 3.27 ms per loop


* Actual compute time is **a lot faster**
* PCI-express transfer overhead

#### Tips
If you have a sequence of ufuncs to apply, pin the data on the GPU by manual transfer

# Accelerate CUDA Libraries Bindings

- Access to CUDA libraries 
- Work seamless with NumPy
    - auto memory transfer
    - managed memory

- cuBLAS: CUDA version of BLAS
- cuSparse: CUDA sparse matrix support
- cuFFT: FFT on CUDA
- cuRAND: random number generation on CUDA

-----

# Low-Level Approach: @numba.cuda.jit

- Numba can generate CUDA functions with the `@jit` decorator
- Decorated function follows CUDA execution model 

## CUDA Execution Model

- Kernel functions
    - visible to the host CPU
    - cannot return any value
        - use output argument
    - associates to a _grid_
- Grid
    - a group of blocks
    - 1D, 2D, 3D
- Blocks
    - a group of threads
    - 1D, 2D, 3D  
- Every thread executes the same kernel
    - thread can use the grid, block, thread coordinate system to determine its ID

In [33]:
Image(url='http://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/grid-of-thread-blocks.png')

## Compiling a CUDA Kernel

In [34]:
from numba import cuda

@numba.cuda.jit("void(float32[:], float32[:], float32[:])")
def vadd(arr_a, arr_b, arr_out):
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x    # number of threads per block
    i = tx + bx * bw
    if i >= arr_out.size:
        return
    arr_out[i] = arr_a[i] + arr_b[i]

### Code Explained

#### Define a CUDA kernel with three 1D float32 arrays as args

```
@numba.cuda.jit("void(float32[:], float32[:], float32[:])")
def vadd(arr_a, arr_b, arr_out):
```

#### Map thread, block coordinate to global position
```
    tx = cuda.threadIdx.x   # thread label (along x dimension)
    bx = cuda.blockIdx.x    # block label (along x dimension)
    bw = cuda.blockDim.x    # number of threads in each block (along x dimension)
    i = tx + bx * bw        # flattened linear address for each thread
```
or simplified to:
```
    i = cuda.grid(1)
``` 

#### Ensure global position is within array size

```
    if i >= arr_out.size:
        return
```

#### The actual work

```
    arr_out[i] = arr_a[i] + arr_b[i]
```

## Launch kernel

#### Prepare data

In [35]:
n = 100
a = np.arange(n, dtype=np.float32)
b = np.arange(n, dtype=np.float32)
c = np.empty_like(a)                 # Must prepare the output array to hold the result


#### Calculate thread, block count

- thread count is set to **warp size** of the GPU
    - Warp size is similar to SIMD vector width on the CPU
    - **performance tips**: set thread count to multiple of warp size
- block count is ceil(n/thread_ct)

**Note:**
This will launch more threads than there are elements in the array

In [36]:
thread_ct = my_gpu.WARP_SIZE
block_ct = int(math.ceil(float(n) / thread_ct))

print("Threads per block:", thread_ct)
print("Block per grid:", block_ct)

Threads per block: 32
Block per grid: 4


#### Launch kernel

Kernel function object uses the ``__getitem__`` (indexing notation) to configure the grid and block dimensions.

```
    kernel_function[griddim, blockdim](*args)
```

- **griddim**
    - Number of blocks per grid (grid dimension)
    - type: int for 1d or 1,2,3-tuple of ints for 1d, 2d, or 3d respectively

- **blockdim**: 
    - Number of threads per block (blockdim dimension)
    - type: int for 1d or 1,2,3-tuple of ints for 1d, 2d, or 3d respectively


In [37]:
vadd[block_ct, thread_ct](a, b, c)    # Last argument is the output array in this case
print(c)

[   0.    2.    4.    6.    8.   10.   12.   14.   16.   18.   20.   22.
   24.   26.   28.   30.   32.   34.   36.   38.   40.   42.   44.   46.
   48.   50.   52.   54.   56.   58.   60.   62.   64.   66.   68.   70.
   72.   74.   76.   78.   80.   82.   84.   86.   88.   90.   92.   94.
   96.   98.  100.  102.  104.  106.  108.  110.  112.  114.  116.  118.
  120.  122.  124.  126.  128.  130.  132.  134.  136.  138.  140.  142.
  144.  146.  148.  150.  152.  154.  156.  158.  160.  162.  164.  166.
  168.  170.  172.  174.  176.  178.  180.  182.  184.  186.  188.  190.
  192.  194.  196.  198.]


# Example: Matrix Matrix Multiplication

- Show manual caching with shared memory
- Not the best matrix matrix multiplication implementation

#### Prepare constants

In [38]:
from numba import float32

bpg = 150
tpb = 32
n = bpg * tpb
shared_mem_size = (tpb, tpb)
griddim = bpg, bpg
blockdim = tpb, tpb

#### Naive version

In [39]:
Image(url="http://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/matrix-multiplication-without-shared-memory.png")

In [40]:
@numba.cuda.jit("void(float32[:,:], float32[:,:], float32[:,:])")
def naive_matrix_mult(A, B, C):
    x, y = cuda.grid(2)
    if x >= n or y >= n:
        return

    C[y, x] = 0
    for i in range(n):
        C[y, x] += A[y, i] * B[i, x]


#### Optimized version (shared memory + blocking)

In [41]:
Image(url="http://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/matrix-multiplication-with-shared-memory.png")

In [42]:
@numba.cuda.jit("void(float32[:,:], float32[:,:], float32[:,:])")
def optimized_matrix_mult(A, B, C):
    # Declare shared memory
    sA = cuda.shared.array(shape=shared_mem_size, dtype=float32)
    sB = cuda.shared.array(shape=shared_mem_size, dtype=float32)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    x, y = cuda.grid(2)

    acc = 0
    for i in range(bpg):
        if x < n and y < n:
            # Prefill cache
            sA[ty, tx] = A[y, tx + i * tpb]
            sB[ty, tx] = B[ty + i * tpb, x]

        # Synchronize all threads in the block
        cuda.syncthreads()

        if x < n and y < n:
            # Compute product
            for j in range(tpb):
                acc += sA[ty, j] * sB[j, tx]

        # Wait until all threads finish the computation
        cuda.syncthreads()

    if x < n and y < n:
        C[y, x] = acc

#### Prepare data

In [43]:
# Prepare data on the CPU
A = np.array(np.random.random((n, n)), dtype=np.float32)
B = np.array(np.random.random((n, n)), dtype=np.float32)

print("N = %d x %d" % (n, n))

# Prepare data on the GPU
dA = cuda.to_device(A)
dB = cuda.to_device(B)
dC = cuda.device_array_like(A)

N = 4800 x 4800


#### Benchmark

In [53]:
# Time the unoptimized version
import time
s = time.time()
naive_matrix_mult[griddim, blockdim](dA, dB, dC)
numba.cuda.synchronize()
e = time.time()
unopt_ans = dC.copy_to_host()
tcuda_unopt = e - s

# Time the optimized version
s = time.time()
optimized_matrix_mult[griddim, blockdim](dA, dB, dC)
numba.cuda.synchronize()
e = time.time()
opt_ans = dC.copy_to_host()
tcuda_opt = e - s

#### Result

In [54]:
assert np.allclose(unopt_ans, opt_ans)
print("Without shared memory:", "%.2f" % tcuda_unopt, "s")
print("With shared memory:", "%.2f" % tcuda_opt, "s")

Without shared memory: 3.86 s
With shared memory: 1.49 s


# Summary

- Numba
    - opensource low-level GPU support
    - CUDA kernel ``@numba.cuda.jit``
    - vectorize ``@numba.vectorize``
    - guvectorize ``@numba.guvectorize``
- Accelerate
    - CUDA libraries ``accelerate.cuda.*``


In [55]:
import numpy as np
from numba import jit, cuda, autojit, float32, float64, f4, f8
from timeit import default_timer as timer

dtype = np.float64
n = 16384
blksize = 128
eps = 1.e-2

In [56]:
def induced_velocity3(x, xvort, gam, vel):
    nx = x.shape[0]
    nvort = xvort.shape[0]
    for i in range(nx):
        vel[i,0] = 0.
        vel[i,1] = 0.
        for j in range(nvort):
            rsq = (x[i,0]-xvort[j,0])**2 + (x[i,1]-xvort[j,1])**2 + eps**2
            vel[i,0] += gam[j] * (x[i,1]-xvort[j,1]) / rsq
            vel[i,1] += -gam[j] * (x[i,0]-xvort[j,0]) / rs
            
def induced_velocity_single(x, xvort, gam):
    r = x - xvort
    rsq = np.sum(r * r, 1) + eps**2
    vel = np.transpose(np.array([r[:,1], -r[:,0]]))
    vel = gam * vel / rsq[:,np.newaxis]
    return vel

def induced_velocity(x, xvort, gam, vel):
    vel[:] = 0.
    for xv, g in zip(xvort, gam):
        r = x - xv
        rsq = np.sum(r * r, 1) + eps**2
        vel += g * np.transpose(np.array([r[:,1], -r[:,0]])) / rsq[:, np.newaxis]


@jit('void(f8[:,:], f8[:,:], f8[:], f8[:,:])', target='cpu')
def r_squared(x1, x2, xv1, xv2):
    return (x1-xv1)**2 + (x2-xv2)**2 + eps**2

@autojit(target='cpu')
def induced_velocity2(x, xvort, gam, vel):
    nx = x.shape[0]
    nvort = xvort.shape[0]
    for i in range(nx):
        vel[i,0] = 0.
        vel[i,1] = 0.
        for j in range(nvort):
            rsq = (x[i,0]-xvort[j,0])**2 + (x[i,1]-xvort[j,1])**2 + eps**2
            vel[i,0] += gam[j] * (x[i,1]-xvort[j,1]) / rsq
            vel[i,1] += -gam[j] * (x[i,0]-xvort[j,0]) / rsq


@jit('void(f8[:,:], f8[:,:], f8[:], f8[:,:])', target='cpu')
def induced_velocity2_test(x, xvort, gam, vel):
    # eps = 1.e-2
    nx = x.shape[0]
    nvort = xvort.shape[0]
    for i in range(nx):
        vel[i,0] = 0.
        vel[i,1] = 0.
        for j in range(nvort):
            rsq = r_squared(x[i,0], x[i,1], xvort[j,0], xvort[j,1])
            vel[i,0] += gam[j] * (x[i,1]-xvort[j,1]) / rsq
            vel[i,1] += -gam[j] * (x[i,0]-xvort[j,0]) / rsq


@cuda.jit('f8[:,:], f8[:,:], f8[:], f8[:,:]')
def induced_velocity3(x, xvort, gam, vel):
    # eps = float32(1.e-2)
    # i, j = cuda.grid(2)
    i = cuda.grid(1)
    if i < x.shape[0]:
        vel[i,0] = float32(0.)
        vel[i,1] = float32(0.)
        nvort = xvort.shape[0]
        for j in range(nvort):
            rsq = (x[i,0]-xvort[j,0])**2 + (x[i,1]-xvort[j,1])**2 + eps**2
            vel[i,0] += gam[j] * (x[i,1]-xvort[j,1]) / rsq
            vel[i,1] += -gam[j] * (x[i,0]-xvort[j,0]) / rsq

@cuda.jit('f8[:,:], f8[:,:], f8[:], f8[:,:]')
def induced_velocity4(x, xvort, gam, vel):
    smem = cuda.shared.array((blksize, 3), dtype=f8)
    t = cuda.threadIdx.x
    i = cuda.grid(1)
    # eps = 1.e-2
    nvort = xvort.shape[0]
    nx = x.shape[0]
    if i < nx:
        x0 = x[i,0]
        x1 = x[i,1]
    xvel = 0
    yvel = 0
    nvort = xvort.shape[0]
    for blk in range((nvort - 1) // blksize + 1):
        # load vortex positions and strengths into shared memory
        j = blk * blksize + t
        if j < nvort:
            smem[t,0] = xvort[j,0]
            smem[t,1] = xvort[j,1]
            smem[t,2] = gam[j]
        else:
            smem[t,0] = 0
            smem[t,1] = 0
            smem[t,2] = 0
        cuda.syncthreads()

        # compute the contributions to the velocity
        for k in range(blksize):
            rsq = (x0-smem[k,0])**2 + (x1-smem[k,1])**2 + eps**2
            xvel +=  smem[k,2] * (x1-smem[k,1]) / rsq
            yvel += -smem[k,2] * (x0-smem[k,0]) / rsq
        cuda.syncthreads()
    if i < nx:
        vel[i,0] = xvel
        vel[i,1] = yvel


In [57]:
def main():
    vort = np.array(np.random.rand(2*n), dtype=dtype).reshape((n,2))
    gamma = np.array(np.random.rand(n), dtype=dtype)
    vel = np.zeros_like(vort)
    start = timer()
    induced_velocity(vort, vort, gamma, vel)
    numpy_time = timer() - start
    print("n = %d" % n)
    print("Numpy".center(40, "="))
    print("Time: %f seconds" % numpy_time)

    vel2 = np.zeros_like(vort)
    start = timer()
    induced_velocity2(vort, vort, gamma, vel2)
    numba_time = timer() - start
    print("Numba".center(40, "="))
    print("Time: %f seconds" % numba_time)
    error = np.max(np.max(np.abs(vel2 - vel)))
    print("Difference: %f" % error)
    print("Speedup: %f" % (numpy_time / numba_time))

    stream = cuda.stream()
    d_vort = cuda.to_device(vort, stream)
    d_gamma = cuda.to_device(gamma, stream)
    vel3 = np.zeros_like(vort)
    d_vel = cuda.to_device(vel3, stream)
    # blockdim = (32,32)
    # griddim = (n // blockdim[0], n // blockdim[1])
    griddim = (n - 1) // blksize + 1
    start = timer()
    induced_velocity3[griddim, blksize, stream](d_vort, d_vort, d_gamma, d_vel)
    d_vel.to_host(stream)
    gpu_time = timer() - start
    error = np.max(np.max(np.abs(vel3 - vel)))
    print("GPU".center(40, "="))
    print("Time: %f seconds" % gpu_time)
    print("Difference: %f" % error)
    print("Speedup: %f" % (numpy_time / gpu_time))
    # print(vel3)

    vel4 = np.zeros_like(vort)
    d_vel2 = cuda.to_device(vel4, stream)
    start = timer()
    induced_velocity4[griddim, blksize, stream](d_vort, d_vort, d_gamma, d_vel2)
    d_vel2.to_host(stream)
    gpu2_time = timer() - start
    error = np.max(np.max(np.abs(vel4 - vel)))
    print("GPU smem".center(40, "="))
    print("Time: %f seconds" % gpu2_time)
    print("Difference: %f" % error)
    print("Speedup: %f" % (numpy_time / gpu2_time))
    # print("Expected".center(40,'-'))
    # print(vel)
    # print("GPU".center(40,'-'))
    # print(vel4)
    # print("Difference".center(40,'-'))
    # print(vel4 - vel)

if __name__ == "__main__":
    main()


n = 16384
=================Numpy==================
Time: 15.638716 seconds
=================Numba==================
Time: 3.346818 seconds
Difference: 0.000000
Speedup: 4.672712
==================GPU===================
Time: 0.069850 seconds
Difference: 0.000000
Speedup: 223.890095
================GPU smem================
Time: 0.018489 seconds
Difference: 0.000000
Speedup: 845.844238
